In [1]:
import torch
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM

# Make sure the script can be imported
# Adjust the path if your notebook is not in the project root
sys.path.append('/root/obfuscation/') 

from reinforce.train_reinforce import BatchThinkingTokenBudgetProcessor

model_name = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16", device_map="auto")

In [9]:
tokenizer.encode("</think>", add_special_tokens=False)

[151668]

In [17]:
tokenizer.decode(151644)

'<|im_start|>'

In [21]:
batch_size = 10

def run_test(messages, processor):
    tokens = tokenizer.apply_chat_template(
        messages, 
        tokenize=True, 
        add_generation_prompt=True, 
        enable_thinking=True, 
        return_tensors="pt"
        ).to(model.device).repeat(batch_size, 1)

    outputs = model.generate(
        tokens, 
        temperature=1.0,
        max_new_tokens=30, 
        logits_processor=[processor], 
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True
    )
    for i in range(batch_size):
        print("--"*100)
        print(tokenizer.decode(outputs[i]))

messages = [{"role": "user", "content": "What is the capital of France?"}]

processor = BatchThinkingTokenBudgetProcessor(tokenizer, min_thinking_tokens=0, max_thinking_tokens=16, batch_size=batch_size)
print(run_test(messages, processor))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
<think>
Okay, the user is asking for the capital of France
</think>

The capital of France is Paris.<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
<think>
Okay, the user is asking for the capital of France
</think>

The capital of France is Paris.<|im_end|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
------------------------------------------------------------